In [25]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load data
data = pd.read_csv('encode_data.csv')  

data

,Quận,Loại nhà,Diện tích (m²),Mức giá (tỷ),Số phòng ngủ,Số toilet,Loại Nhà (mã hóa),Quận (mã hóa),Khoảng giá,Mức giá (triệu/m²)
0,Gia Lâm,Căn hộ chung cư,55.0,2.07,2,1,0,4,1,37.636364
1,Nam Từ Liêm,Căn hộ chung cư,140.0,9.50,4,3,0,12,1,67.857143
2,Hà Đông,Căn hộ chung cư,98.0,3.19,3,2,0,9,1,32.551020
3,Nam Từ Liêm,Căn hộ chung cư,59.0,2.90,2,2,0,12,1,49.152542
4,Hà Đông,Căn hộ chung cư,92.0,3.50,2,2,0,9,1,38.043478
...,...,...,...,...,...,...,...,...,...,...
7887,Bắc Từ Liêm,Căn hộ chung cư,98.0,6.35,3,2,0,1,1,64.795918
7888,Hà Đông,Nhà mặt phố,225.0,66.00,6,8,1,9,3,293.333333
7889,Nam Từ Liêm,Nhà riêng,40.0,4.00,4,4,2,12,1,100.000000
7890,Gia Lâm,Căn hộ chung cư,70.0,2.45,2,2,0,4,1,35.000000


In [26]:
X = data[['Quận (mã hóa)', 'Loại Nhà (mã hóa)', 'Diện tích (m²)', 'Số phòng ngủ', 'Số toilet', 'Khoảng giá']]
y = data['Mức giá (tỷ)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [27]:
lr = LinearRegression()

In [28]:
lr.fit(X_train, y_train)

LinearRegression()

In [29]:
y_pred_lr = lr.predict(X_test)

mean_squared_error(y_test, y_pred_lr)

137.1357375268151

In [30]:
RFR = RandomForestRegressor(random_state=13)

In [31]:
param_grid_RFR = {
    'max_depth': [12, 15, 18],
    'n_estimators': [200, 400, 500],
    'min_samples_split': [9, 12, 15]
}

In [32]:
rfr_cv = GridSearchCV(RFR, param_grid_RFR, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
                                                  
rfr_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=13), n_jobs=-1,
             param_grid={'max_depth': [12, 15, 18],
                         'min_samples_split': [9, 12, 15],
                         'n_estimators': [200, 400, 500]},
             scoring='neg_mean_squared_error')

In [36]:
results = rfr_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng truy cập
df_results = pd.DataFrame(results)

# In các tham số và best_score_ cho mỗi cấu hình tham số
for index, row in df_results.iterrows():
    params = row['params']
    mean_test_score = row['mean_test_score']
    std_test_score = row['std_test_score']
    rmse_score = np.sqrt(-mean_test_score)
    print(f"Parameters: {params}, RMSE: {rmse_score:.4f}, Std: {std_test_score:.4f}")

# In ra best_score_ tổng thể
y_pred = rfr_cv.best_estimator_.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best MAE: {mae:.4f}")
print(f"Best MSE: {mse:.4f}")
print(f"Best R-squared score: {r_squared:.4f}")
print(f"Best RMSE: {rmse:.4f}")
print("Best Parameters: ", rfr_cv.best_params_)

Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 200}, RMSE: 11.3512, Std: 62.0091
Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 400}, RMSE: 11.3756, Std: 62.3556
Parameters: {'max_depth': 12, 'min_samples_split': 9, 'n_estimators': 500}, RMSE: 11.3728, Std: 62.3755
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 200}, RMSE: 11.1357, Std: 56.2932
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 400}, RMSE: 11.1068, Std: 55.1557
Parameters: {'max_depth': 12, 'min_samples_split': 12, 'n_estimators': 500}, RMSE: 11.1064, Std: 55.2492
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 200}, RMSE: 11.0234, Std: 51.2492
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 400}, RMSE: 10.9815, Std: 51.0463
Parameters: {'max_depth': 12, 'min_samples_split': 15, 'n_estimators': 500}, RMSE: 10.9683, Std: 50.9171
Parameters: {'max_depth': 15, 'min_samples_split': 9, 'n_e

In [37]:
ridge = Ridge()

In [38]:
param_grid_ridge = {
    'alpha': [10, 12, 14, 16, 18],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag']
}

In [39]:
ridge_cv = GridSearchCV(ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [40]:
ridge_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [10, 12, 14, 16, 18],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg', 'sag']},
             scoring='neg_mean_squared_error')

In [ ]:
# Lấy kết quả của mỗi lần thử
results = ridge_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng hiển thị
results_df = pd.DataFrame(results)

# In ra các cột quan trọng
print(results_df[['mean_test_score', 'std_test_score', 'param_alpha', 'param_solver']])

# In ra mô hình tốt nhất và các tham số tương ứng
print(f"Best parameters found: {ridge_cv.best_params_}")
print(f"Best mean squared error: {np.sqrt(-ridge_cv.best_score_):.4f}")

    mean_test_score  std_test_score param_alpha param_solver
0       -123.583779       36.630259          10         auto
1       -123.583779       36.630259          10          svd
2       -123.583779       36.630259          10     cholesky
3       -123.572141       36.643722          10         lsqr
4       -123.571509       36.644454          10    sparse_cg
5       -126.512701       40.906456          10          sag
6       -123.582032       36.653003          12         auto
7       -123.582032       36.653003          12          svd
8       -123.582032       36.653003          12     cholesky
9       -123.570448       36.666399          12         lsqr
10      -123.569689       36.667277          12    sparse_cg
11      -126.512765       40.907332          12          sag
12      -123.580452       36.675572          14         auto
13      -123.580452       36.675572          14          svd
14      -123.580452       36.675572          14     cholesky
15      -123.568922     

In [41]:
y_pred = ridge_cv.best_estimator_.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best MAE: {mae:.4f}")
print(f"Best MSE: {mse:.4f}")
print(f"Best R-squared score: {r_squared:.4f}")
print(f"Best RMSE: {rmse:.4f}")
print("Best Parameters: ", rfr_cv.best_params_)

Best MAE: 6.2459
Best MSE: 137.0737
Best R-squared score: 0.5738
Best RMSE: 11.7078
Best Parameters:  {'max_depth': 15, 'min_samples_split': 15, 'n_estimators': 500}


In [42]:
GBR = GradientBoostingRegressor()

In [43]:
param_grid_GBR = {
    'max_depth': [12, 15],
    'n_estimators': [100, 200],
    'min_samples_leaf': [10, 15],
    'learning_rate': [0.001, 0.01, 0.1],
    'max_features': [0.01, 0.1]
}

In [44]:
GBR_cv = GridSearchCV(GBR, param_grid_GBR, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

In [45]:
GBR_cv.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [12, 15], 'max_features': [0.01, 0.1],
                         'min_samples_leaf': [10, 15],
                         'n_estimators': [100, 200]},
             scoring='neg_mean_squared_error')

In [47]:
# Lấy kết quả của mỗi lần thử
results = GBR_cv.cv_results_

# Chuyển đổi kết quả thành DataFrame để dễ dàng hiển thị
results_df = pd.DataFrame(results)

# In ra các cột quan trọng
print(results_df[['mean_test_score', 'std_test_score', 'param_max_depth', 'param_n_estimators', 
                  'param_min_samples_leaf', 'param_learning_rate', 'param_max_features']])

# In ra mô hình tốt nhất và các tham số tương ứng
print(f"Best parameters found: {GBR_cv.best_params_}")
print(f"Best RMSE: {np.sqrt(-GBR_cv.best_score_):.4f}")

    mean_test_score  std_test_score param_max_depth param_n_estimators  \
0       -442.971677      120.875507              12                100   
1       -391.044642      114.078165              12                200   
2       -445.646148      121.492578              12                100   
3       -395.867788      114.614863              12                200   
4       -443.410307      121.364876              12                100   
5       -391.755007      114.467758              12                200   
6       -446.338217      121.246521              12                100   
7       -395.454413      114.351515              12                200   
8       -443.498438      120.950912              15                100   
9       -391.465743      114.098680              15                200   
10      -446.047062      121.757212              15                100   
11      -396.196952      114.748123              15                200   
12      -443.836025      120.907232   

In [48]:
y_pred = GBR_cv.best_estimator_.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best MAE: {mae:.4f}")
print(f"Best MSE: {mse:.4f}")
print(f"Best R-squared score: {r_squared:.4f}")
print(f"Best RMSE: {rmse:.4f}")
print("Best Parameters: ", rfr_cv.best_params_)

Best MAE: 2.4261
Best MSE: 59.3638
Best R-squared score: 0.8154
Best RMSE: 7.7048
Best Parameters:  {'max_depth': 15, 'min_samples_split': 15, 'n_estimators': 500}


In [52]:
# Tạo mô hình VotingRegressor kết hợp cả hai mô hình GradientBoostingRegressor và RandomForestRegressor
voting_regressor = VotingRegressor(estimators=[('gbr', GBR_cv.best_estimator_), ('rfr', rfr_cv.best_estimator_)],
                                   weights=[3,1])

In [53]:
voting_regressor.fit(X_train, y_train)

VotingRegressor(estimators=[('gbr',
                             GradientBoostingRegressor(max_depth=12,
                                                       max_features=0.1,
                                                       min_samples_leaf=15)),
                            ('rfr',
                             RandomForestRegressor(max_depth=15,
                                                   min_samples_split=15,
                                                   n_estimators=500,
                                                   random_state=13))],
                weights=[3, 1])

In [54]:
# Dự đoán trên tập kiểm tra
y_pred = voting_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best MAE: {mae:.4f}")
print(f"Best MSE: {mse:.4f}")
print(f"Best R-squared score: {r_squared:.4f}")
print(f"Best RMSE: {rmse:.4f}")
print("Best Parameters: ", rfr_cv.best_params_)

Best MAE: 2.4136
Best MSE: 71.7107
Best R-squared score: 0.7770
Best RMSE: 8.4682
Best Parameters:  {'max_depth': 15, 'min_samples_split': 15, 'n_estimators': 500}


In [55]:
from sklearn.ensemble import AdaBoostRegressor

In [56]:
ada_boost_gbr = AdaBoostRegressor(base_estimator=GBR_cv.best_estimator_, n_estimators=50, random_state=42)

In [57]:
ada_boost_gbr.fit(X_train, y_train)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


AdaBoostRegressor(base_estimator=GradientBoostingRegressor(max_depth=12,
                                                           max_features=0.1,
                                                           min_samples_leaf=15),
                  random_state=42)

In [58]:
# Dự đoán trên tập kiểm tra
y_pred_ada_boost_gbr = ada_boost_gbr.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Best MAE: {mae:.4f}")
print(f"Best MSE: {mse:.4f}")
print(f"Best R-squared score: {r_squared:.4f}")
print(f"Best RMSE: {rmse:.4f}")
print("Best Parameters: ", rfr_cv.best_params_)

Best MAE: 2.4136
Best MSE: 71.7107
Best R-squared score: 0.7770
Best RMSE: 8.4682
Best Parameters:  {'max_depth': 15, 'min_samples_split': 15, 'n_estimators': 500}
